In [1]:
!pip install ultralytics

   ---------------------------------------- 0.0/1.0 MB ? eta -:--:--
   ------------------------------ --------- 0.8/1.0 MB 4.2 MB/s eta 0:00:01
   ---------------------------------------- 1.0/1.0 MB 3.3 MB/s eta 0:00:00
   ---------------------------------------- 0.0/11.3 MB ? eta -:--:--
   ---- ----------------------------------- 1.3/11.3 MB 6.1 MB/s eta 0:00:02
   ---------- ----------------------------- 2.9/11.3 MB 7.3 MB/s eta 0:00:02
   ----------------- ---------------------- 5.0/11.3 MB 7.9 MB/s eta 0:00:01
   ---------------------- ----------------- 6.3/11.3 MB 7.6 MB/s eta 0:00:01
   -------------------------- ------------- 7.6/11.3 MB 7.1 MB/s eta 0:00:01
   ------------------------------------ --- 10.2/11.3 MB 8.0 MB/s eta 0:00:01
   ---------------------------------------- 11.3/11.3 MB 8.1 MB/s eta 0:00:00
   ---------------------------------------- 0.0/216.1 MB ? eta -:--:--
   ---------------------------------------- 2.4/216.1 MB 11.2 MB/s eta 0:00:20
    --------------


[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
import cv2
import mediapipe as mp
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_face_mesh = mp.solutions.face_mesh
mp_hands = mp.solutions.hands
mp_pose = mp.solutions.pose

In [2]:
# Drawing spec opsional
drawing_spec = mp_drawing.DrawingSpec(thickness=1, circle_radius=1)

# Buka kamera
cap = cv2.VideoCapture(0)

# Jalankan semua model sekaligus
with mp_face_mesh.FaceMesh(
    max_num_faces=1,
    refine_landmarks=True,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5) as face_mesh, \
  mp_hands.Hands(
    model_complexity=0,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5) as hands, \
  mp_pose.Pose(
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5) as pose:

    while cap.isOpened():
        success, image = cap.read()
        if not success:
            print("Ignoring empty camera frame.")
            continue

        # Proses gambar
        image = cv2.flip(image, 1)
        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image_rgb.flags.writeable = False

        # Deteksi semua model
        face_results = face_mesh.process(image_rgb)
        hand_results = hands.process(image_rgb)
        pose_results = pose.process(image_rgb)

        image_rgb.flags.writeable = True
        image = cv2.cvtColor(image_rgb, cv2.COLOR_RGB2BGR)

        # Face Mesh
        if face_results.multi_face_landmarks:
            for face_landmarks in face_results.multi_face_landmarks:
                mp_drawing.draw_landmarks(
                    image=image,
                    landmark_list=face_landmarks,
                    connections=mp_face_mesh.FACEMESH_TESSELATION,
                    landmark_drawing_spec=None,
                    connection_drawing_spec=mp_drawing_styles.get_default_face_mesh_tesselation_style())
                mp_drawing.draw_landmarks(
                    image=image,
                    landmark_list=face_landmarks,
                    connections=mp_face_mesh.FACEMESH_CONTOURS,
                    landmark_drawing_spec=None,
                    connection_drawing_spec=mp_drawing_styles.get_default_face_mesh_contours_style())
                mp_drawing.draw_landmarks(
                    image=image,
                    landmark_list=face_landmarks,
                    connections=mp_face_mesh.FACEMESH_IRISES,
                    landmark_drawing_spec=None,
                    connection_drawing_spec=mp_drawing_styles.get_default_face_mesh_iris_connections_style())

        # Hand Detection
        if hand_results.multi_hand_landmarks:
            for hand_landmarks in hand_results.multi_hand_landmarks:
                mp_drawing.draw_landmarks(
                    image,
                    hand_landmarks,
                    mp_hands.HAND_CONNECTIONS,
                    mp_drawing_styles.get_default_hand_landmarks_style(),
                    mp_drawing_styles.get_default_hand_connections_style())

        # Pose Detection
        if pose_results.pose_landmarks:
            mp_drawing.draw_landmarks(
                image,
                pose_results.pose_landmarks,
                mp_pose.POSE_CONNECTIONS,
                mp_drawing_styles.get_default_pose_landmarks_style())

        # Tampilkan
        cv2.imshow('Multi Model Detection', image)
        if cv2.waitKey(5) & 0xFF == 27:
            break

cap.release()
cv2.destroyAllWindows()

KeyboardInterrupt: 

In [ ]:
import cv2
from ultralytics import YOLO

# --- Muat Model YOLO ---
# 1. Model untuk Pose Estimation (akan diunduh otomatis)
pose_model = YOLO('models/yolov8n-pose.pt')

# 2. Model untuk Hand Gesture (GANTI DENGAN PATH MODEL ANDA)
# CATATAN: Anda harus mencari dan mengunduh model deteksi gestur tangan.
# Contoh model bisa dicari di Roboflow Universe.
# Jika hanya ingin deteksi tangan (bukan gestur), gunakan model deteksi objek biasa.
try:
    hand_gesture_model = YOLO('models/yolov8n-hand.pt') # Placeholder, ganti dengan model Anda
except Exception as e:
    print("Peringatan: Model gestur tangan tidak ditemukan. Bagian ini akan dilewati.")
    print("Unduh model dari Roboflow atau sumber lain dan letakkan sebagai 'yolov8n_hand_gesture.pt'")
    hand_gesture_model = None

# 3. Model untuk Face Detection (GANTI DENGAN PATH MODEL ANDA)
# CATATAN: 'yolov8n-face.pt' adalah model populer dari komunitas.
# Anda perlu mengunduhnya secara manual.
try:
    face_model = YOLO('models/yolov8n-face.pt') # Placeholder, ganti dengan model Anda
except Exception as e:
    print("Peringatan: Model deteksi wajah tidak ditemukan. Bagian ini akan dilewati.")
    print("Unduh model 'yolov8n-face.pt' dan letakkan di folder yang sama.")
    face_model = None


# --- Buka Kamera ---
cap = cv2.VideoCapture(0)
if not cap.isOpened():
    print("Error: Tidak bisa membuka kamera.")
    exit()

while True:
    success, frame = cap.read()
    if not success:
        print("Ignoring empty camera frame.")
        break

    # Balik gambar secara horizontal untuk tampilan seperti cermin
    frame = cv2.flip(frame, 1)
    
    # Buat salinan frame untuk setiap model agar hasil tidak tumpang tindih secara mentah
    annotated_frame = frame.copy()

    # --- Jalankan Deteksi ---

    # 1. Deteksi Pose
    # `verbose=False` agar tidak mencetak log ke konsol setiap frame
    pose_results = pose_model(frame, verbose=False)
    # Gunakan method plot() bawaan dari ultralytics untuk menggambar hasil
    annotated_frame = pose_results[0].plot()

    # 2. Deteksi Gestur Tangan (jika model ada)
    if hand_gesture_model:
        # Jalankan pada frame yang sudah ada anotasi pose
        hand_results = hand_gesture_model(annotated_frame, verbose=False)
        # Gambar hasil deteksi tangan di atasnya
        annotated_frame = hand_results[0].plot()

    # 3. Deteksi Wajah (jika model ada)
    if face_model:
        # Jalankan pada frame yang sudah ada anotasi pose dan tangan
        face_results = face_model(annotated_frame, verbose=False)
        # Gambar hasil deteksi wajah di atasnya
        annotated_frame = face_results[0].plot()

    # Tampilkan hasil akhir
    cv2.imshow('YOLO Multi-Model Detection', annotated_frame)

    # Keluar dari loop jika tombol 'ESC' ditekan (ASCII 27)
    if cv2.waitKey(1) & 0xFF == 27:
        break

# Lepaskan sumber daya
cap.release()
cv2.destroyAllWindows()

Peringatan: Model gestur tangan tidak ditemukan. Bagian ini akan dilewati.
Unduh model dari Roboflow atau sumber lain dan letakkan sebagai 'yolov8n_hand_gesture.pt'
